In [ ]:
"""
Visualisation des capacités de production d'électricité régionales par filière au cours de  la période 2013 à 2022 en France mùétropolitaine.

Created on Tue Sep 10 15:58:43 2024

@author: Thierry ALLEM
"""

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import geopandas as gpd
import folium
from folium import plugins
import numpy as np
import seaborn as sns
import matplotlib.patches as patches
import matplotlib.dates as mdates

In [ ]:
import warnings
# Pour ignorer tous les warnings
warnings.filterwarnings('ignore')

In [ ]:
# AFFICHAGES DE GRAPHIQUES DES CAPACITES DE PRODUCTION

In [ ]:
# Importation du fichier eco2mix_regional_def préparé, incluant les tco,  TCH et cmax_rte
df_cap_prod = pd.read_csv("df_eco2_tch_rte_extrapole.csv",sep = ',', encoding='latin-1')

df_cap_prod.head()

In [ ]:
df_cap_prod.info()

In [ ]:
# ========================================= LES CAPACITES MAXIMALES ========================================================

In [ ]:
# Répartition des principales capacités de production en France - THERMIQUE, NUCLEAIRE et HYDRAULIQUE - Carte de France interactive

In [ ]:
df_geo_prod_tnh = pd.read_csv('Repartition_principales_installations_de_production_hors_solaire_et_C.csv', sep=';', encoding='latin-1')

df_geo_prod_tnh['Capacite (MW)'] = pd.to_numeric(df_geo_prod_tnh['Capacite (MW)'], errors='coerce')

In [ ]:
# Chargement  des contours des régions administratives françaises
gdf_regions_france = gpd.read_file('https://github.com/gregoiredavid/france-geojson/raw/master/regions.geojson')

# Préparation de la carte avec folium
m = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

# Ajout des contours des régions à la carte
folium.GeoJson(
    gdf_regions_france,
    name='regions',
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'black',
        'weight': 1
    }
).add_to(m)

# Ajout des noms des régions
for _, region in gdf_regions_france.iterrows():
    folium.Marker(
        location=[region.geometry.representative_point().y, region.geometry.representative_point().x],
        icon=folium.DivIcon(html=f"<div style='font-size: 12pt'>{region['nom']}</div>")
    ).add_to(m)

# Définition d'une palette de couleurs pour les filières
filiere_colors = {
    'Nucléaire': 'gold',
    'Hydraulique': 'blue',
    'Thermique': 'purple'    
}

# # Préparation des données pour affichage de cercles proportionnels à la capacité maximale totale de la filière
for _, row in df_geo_prod_tnh.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=(row['Capacite (MW)']**0.5) / 2.5,  # Pour réduire la taille des cercles
        color=filiere_colors.get(row['Filière'], 'gray'),
        fill=True,
        fill_color=filiere_colors.get(row['Filière'], 'gray'),
        fill_opacity=0.6,
        popup=f"Filière: {row['Filière']}<br>Capacité: {row['Capacite (MW)']} MW"
    ).add_to(m)

# Ajout d'une légende
legend_html = '''
<div style="position: fixed; bottom: 50px; left: 50px; width: 150px; height: 120px; 
     background-color: white; border:2px solid grey; z-index:9999; font-size:14px;">
     &nbsp; Légende <br>
     &nbsp; <i class="fa fa-circle fa-1x" style="color:gold"></i>&nbsp; Nucléaire <br>
     &nbsp; <i class="fa fa-circle fa-1x" style="color:blue"></i>&nbsp; Hydraulique <br>
     &nbsp; <i class="fa fa-circle fa-1x" style="color:purple"></i>&nbsp; Thermique <br>
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Affichage de la carte
m.save('carte_production_therm_nuc_hydro.html')
print("La carte a été sauvegardée dans le fichier 'carte_production.html'.")

In [ ]:
# Répartition des principales capacités de production en France - EOLIEN et SOLAIRE - Carte de France interactive

In [ ]:
# Chargement du fichier CSV
df_geo_prod_eol_sol = pd.read_csv('parc_eolien_solaire.csv', sep=';', encoding='latin-1')
df_geo_prod_eol_sol.head()

In [ ]:
df_geo_prod_eol_sol.info()

In [ ]:
# Recherchez de valeurs manquantes
df_geo_prod_eol_sol.isna().sum()

In [ ]:
# Remplacement des valeurs manquantes par 0
df_geo_prod_eol_sol = df_geo_prod_eol_sol.fillna(0)
# Vérification
df_geo_prod_eol_sol.isna().sum()

In [ ]:
# Calcul des capacités totales par région pour chaque filière
df_eolien = df_geo_prod_eol_sol.groupby('region')['parc_ installe_eolien_MW'].sum().reset_index()
df_solaire = df_geo_prod_eol_sol.groupby('region')['parc_installe_solaire_MW'].sum().reset_index()

In [ ]:
# Conversion des coordonnées en colonnes séparées
df_geo_prod_eol_sol[['lat', 'lon']] = df_geo_prod_eol_sol['geo_point_region'].str.split(',', expand=True).astype(float)

In [ ]:
# Chargement des contours des régions administratives françaises
gdf_regions_france_eol_sol = gpd.read_file('https://github.com/gregoiredavid/france-geojson/raw/master/regions.geojson')

# Préparation de la carte avec folium
m_eol_sol = folium.Map(location=[46.603354, 1.888334], zoom_start=6)


In [ ]:
# Ajout des contours des régions à la carte
folium.GeoJson(
    gdf_regions_france_eol_sol,
    name='regions',
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'black',
        'weight': 1
    }
).add_to(m_eol_sol)

# Ajout des noms des régions
for _, region in gdf_regions_france_eol_sol.iterrows():
    folium.Marker(
        location=[region.geometry.representative_point().y, region.geometry.representative_point().x],
        icon=folium.DivIcon(html=f"<div style='font-size: 12pt'>{region['nom']}</div>")
    ).add_to(m_eol_sol)

# Définition d'une palette de couleurs pour les filières
filiere_colors = {
    'Eolien': 'deepskyblue',
    'Solaire': 'orange'}

# Préparation des données pour affichage de cercles proportionnels à la capacité maximale totalede la filière
for _, row in df_eolien.iterrows():
    region_name = row['region']
    capacity_eolien = row['parc_ installe_eolien_MW']
    coords = df_geo_prod_eol_sol[df_geo_prod_eol_sol['region'] == region_name][['lat', 'lon']].iloc[0]
    
    folium.CircleMarker(
        location=[coords['lat'] + 0.30, coords['lon']],  #  Décalage plus important pour éviter la superposition des cercles
        radius=(capacity_eolien**0.5) / 5,  # Réduction de la taille pour éviter les chevauchements de cercles
        color=filiere_colors['Eolien'],
        fill=True,
        fill_color=filiere_colors['Eolien'],
        fill_opacity=0.6,
        popup=f"Région: {region_name}<br>Capacité éolienne: {capacity_eolien} MW"
    ).add_to(m_eol_sol)

for _, row in df_solaire.iterrows():
    region_name = row['region']
    capacity_solaire = row['parc_installe_solaire_MW']
    coords = df_geo_prod_eol_sol[df_geo_prod_eol_sol['region'] == region_name][['lat', 'lon']].iloc[0]
    
    folium.CircleMarker(
        location=[coords['lat'] - 0.30, coords['lon']],  # Décalage plus important pour éviter la superposition des cercles
        radius=(capacity_solaire**0.5) / 5,  # Réduction de la taille pour éviter les chevauchements de cercles
        color=filiere_colors['Solaire'],
        fill=True,
        fill_color=filiere_colors['Solaire'],
        fill_opacity=0.6,
        popup=f"Région: {region_name}<br>Capacité solaire: {capacity_solaire} MW"
    ).add_to(m_eol_sol)

# Ajout d'une légende
legend_html = '''
<div style="position: fixed; bottom: 50px; left: 50px; width: 150px; height: 120px; 
     background-color: white; border:2px solid grey; z-index:9999; font-size:14px;">
     &nbsp; Légende <br>
     &nbsp; <i class="fa fa-circle fa-1x" style="color:deepskyblue"></i>&nbsp; Eolien <br>
     &nbsp; <i class="fa fa-circle fa-1x" style="color:orange"></i>&nbsp; Solaire <br>
</div>
'''
m_eol_sol.get_root().html.add_child(folium.Element(legend_html))

# Sauvegarde de la carte
m_eol_sol.save('carte_production_eolien_solaire.html')
print("La carte a été sauvegardée dans le fichier 'carte_production_eolien_solaire.html'.")

In [ ]:
# =========================================== CAMENBERTS DES CAPACITES MAXIMALES DE PRODUCTION REGIONALES ANNUELLES=============================

In [ ]:
# Création du répertoire de sauvegarde des graphiques
output_dir = "Camenberts_Cmax_grouped"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Définition des labels pour chacune des filières
filières_labels = ['Thermique', 'Nucleaire', 'Eolien', 'Solaire', 'Hydraulique', 'Bioenergies', 'Solde Export.', 'Solde Import.']
filieres = ['thermique_p_disp', 'nucleaire_p_disp', 'eolien_p_disp', 'solaire_p_disp', 'hydraulique_p_disp',
            'bioenergies_p_disp', 'export', 'import']

# Couleurs personnalisées pour les camemberts
couleurs = ['purple', 'gold', 'aquamarine', 'orange', 'blue', 'green', 'crimson', 'deeppink']

# Liste des régions
regions = df_cap_prod['region'].unique()

# Liste des années
annees = range(2013,2023)

def pie_autopct(pct, all_vals):
    # Je formate les étiquettes afin d'afficher uniquement les 3 principales parts
    if pct >= sorted(all_vals, reverse=True)[2]:  # Garde les 3 principales parts
        return f'{pct:.1f}%'
    else:
        return ''  # POur ne rien afficher pour les parts non principales

# Pour chaque région
for region in regions:
    # Création de la figure et des sous-graphiques (3 colonnes)
    fig, axes = plt.subplots(nrows=(len(annees) + 2) // 3, ncols=3, figsize=(8.27, 11.69))  # A4 size: 8.27x11.69 inches
    axes = axes.flatten()
    
    # Pour chaque année
    for idx, annee in enumerate(annees):
        # Sélection des données pour la région et l'année
        data = df_cap_prod[(df_cap_prod['region'] == region) & (df_cap_prod['annee'] == annee)]
        
        # Calcul de la production annuelle par filière, en TWh
        production_annuelle = data[filieres].abs().sum() / 2 / 1000000
        
        # Calcul des pourcentages
        total_production = production_annuelle.sum()
        pourcentages = 100 * production_annuelle / total_production
        
        # Création du camembert
        wedges, texts, autotexts = axes[idx].pie(production_annuelle, colors=couleurs, shadow=True, startangle=90,
                                                 autopct=lambda pct: pie_autopct(pct, pourcentages))
        
        # Titre de chaque camembert (année)
        axes[idx].set_title(f'{annee}', fontsize=12)
    
    # Suppression des axes non utilisés
    for i in range(len(annees), len(axes)):
        fig.delaxes(axes[i])
    
    # Titre principal
    fig.suptitle(f"Répartition des capacités maximales de production en {region}", fontsize=16)
    
    # Légende
    fig.legend(wedges, filières_labels, title="Filières", loc="lower center", bbox_to_anchor=(0.5, -0.05), ncol=4)
    
    # Ajustement de l'espacement
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    
    # Sauvegarde du graphique
    filename = f"pie_cmax_{region}.png"
    plt.savefig(os.path.join(output_dir, filename), dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
# =========================================== CAMENBERTS DES CAPACITES MAXIMALES DE PRODUCTION METROPOLE ANNUELLES=============================

In [ ]:
def pie_autopct2(pct, all_vals, top_indices):
    total = sum(all_vals)
    val = pct / 100. * total
    # Affiche les % uniquement pour les parts les plus importantes
    if any(idx in top_indices for idx, value in enumerate(all_vals)):
        return f'{pct:.1f}%'
    else:
        return ''  # Ne rien afficher pour les parts non principales

# Création de la figure et des sous-graphiques
fig, axes = plt.subplots(nrows=(len(annees) + 2) // 3, ncols=3, figsize=(8.27, 11.69))  # Format A4
axes = axes.flatten()

# Pour chaque année
for idx, annee in enumerate(annees):
    # Sélection des données pour l'année (ensemble des régions)
    data = df_cap_prod[df_cap_prod['annee'] == annee]
    
    # Calcul de la production annuelle par filière pour toutes les régions, en TWh
    production_annuelle = data[filieres].abs().sum() / 2 / 1000000
    
    # Calcul des pourcentages
    total_production = production_annuelle.sum()
    pourcentages = 100 * production_annuelle / total_production
    
    # Identification des indices des 3 principales parts
    sorted_indices = sorted(range(len(production_annuelle)), key=lambda i: production_annuelle[i], reverse=True)[:3]

    # Création du camembert
    wedges, texts, autotexts = axes[idx].pie(pourcentages, colors=couleurs, shadow=True, startangle=90,
                                             autopct=lambda pct: pie_autopct2(pct, production_annuelle, sorted_indices))
    
    # Titre de chaque camembert
    axes[idx].set_title(f'{annee}', fontsize=12)

    # Amélioration de la lisibilité des textes
    for text in autotexts:
        text.set_color('black')
        text.set_fontsize(10)
    
# Suppression des axes non utilisés
for i in range(len(annees), len(axes)):
    fig.delaxes(axes[i])

# Titre principal
fig.suptitle("Répartition des capacités maximales de production en Métropole", fontsize=16)

# Légende
fig.legend(wedges, filières_labels, title="Filières", loc="lower center", bbox_to_anchor=(0.5, -0.05), ncol=4)

# Ajustement de l'espacement
plt.tight_layout(rect=[0, 0, 1, 0.95])

# Sauvegarde du graphique 
filename = "pie_cmax_metropole.png"
plt.savefig(os.path.join(output_dir, filename), dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# AFFICHAGE DE TOUS LES CAMENBERTS ANNUELS D'UNE REGION DANS UN FICHIER

In [ ]:
# Définition des nouveaux labels pour les filières
filières = ['cmax_thermique_rte', 'cmax_nucleaire_rte', 'cmax_eolien_rte', 'cmax_solaire_rte', 'cmax_hydraulique_rte', 'cmax_bioenergies_rte', 'import']

# Liste des régions
regions = df_cap_prod['region'].unique()

# Liste des années
annees = df_cap_prod['annee'].unique()

# Pour chaque région
for region in regions:
    # Création d'une figure pour la région
    fig, axs = plt.subplots(1, len(annees), figsize=(20, 6))
    fig.suptitle(f'Répartition de la capacité de production maximale annuelle par filière pour la région {region}')
    
    for i, annee in enumerate(annees):
        # Sélection des données pour la région et l'année
        data = df_cap_prod[(df_cap_prod['region'] == region) & (df_cap_prod['annee'] == annee)]
        
        # Calcul de la capacité de production maximale annuelle par filière, en TWh
        production_annuelle = data[filières].sum() / 2 / 1000000
        
        # Calcul des pourcentages
        total_production = production_annuelle.sum()
        pourcentages = 100 * production_annuelle / total_production
        
        # Formatage des labels pour la légende
        legend_labels = [f'{label} ({value:.2f} TWh, {pct:.1f}%)' for label, value, pct in zip(filières, production_annuelle, pourcentages)]
        
        # Création du camembert dans le subplot
        wedges, texts = axs[i].pie(production_annuelle, shadow=True, startangle=90)
        
        # Ajout d'une légende sous le camembert
        axs[i].legend(wedges, legend_labels, title="Filières", loc="upper center", bbox_to_anchor=(0.5, -0.2))
        axs[i].set_title(f'Année {annee}')
    
    plt.tight_layout(rect=[0, 0.1, 1, 0.95])
    plt.show()
    # Sauvegarde du camembert en PNG
    plt.savefig(f'cap_max_annuel_{region}.png', dpi=300)

In [ ]:
# AFFICHAGE DE TOUS LES CAMENBERTS ANNUELS POUR LA METROPOLE DANS UN FICHIER

In [ ]:
for annee in annees:
    # Sélection des données pour l'année
    data = df_cap_prod[df_cap_prod['annee'] == annee]
    
    # Calcul de la production annuelle par filière
    production_annuelle = data[filières].sum() / 2 / 1000000
    
    # Calcul des pourcentages
    total_production = production_annuelle.sum()
    pourcentages = 100 * production_annuelle / total_production
    
    # Formatage des labels pour la légende
    legend_labels = [f'{label} ({value:.2f} TWh, {pct:.1f}%)' for label, value, pct in zip(filières_labels, production_annuelle, pourcentages)]
    
    # Création du camembert
    plt.figure(figsize=(10, 6))
    wedges, texts = plt.pie(production_annuelle, shadow=True, startangle=90, 
                            explode=[0.1 if value < 1 else 0 for value in production_annuelle])
    
    # Ajout d'une légende
    plt.legend(wedges, legend_labels, title="Filières", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
    
    plt.title(f'Répartition de la capacité de production maximale  annuelle par filière pour l\'ensemble des régions, hors Corse, en {annee}')
    plt.show()
    # Sauvegarde du camembert en PNG
    plt.savefig(f'cap_max_annuel_toutes_regions_{annee}.png', dpi=300)

In [ ]:
# *************************      AFFICHAGE DES RELATIONS ENTRES CAPACITES DE PRODUCTION ET POPULATION ************************

In [ ]:
# Chargement du fichier incluant les capacités de production les populations régionales
df_cap_prod_pop = pd.read_csv('df_blackout_ML_pop.csv',sep = ',', encoding='latin-1')
df_cap_prod_pop.head()

In [ ]:
df_cap_prod_pop.info()

In [ ]:
# ++++++++++++++++++++++++          Capacités maximales de production / population, par année et par jour


# Création du répertoire de sortie
output_dir = "Capacites_max_population_jour"
os.makedirs(output_dir, exist_ok=True)

# Calcul du rapport puissance installée / population en W/habitant
df_cap_prod_pop['cap_max_pop_region'] = (df_cap_prod_pop['cap_prod_max_thermique'] +
                                      df_cap_prod_pop['cap_prod_max_nucleaire'] +
                                      df_cap_prod_pop['cap_prod_max_hydraulique'] +
                                      df_cap_prod_pop['eolien_p_disp'] +
                                      df_cap_prod_pop['solaire_p_disp'] +
                                      df_cap_prod_pop['cap_prod_max_bioenergies']) * 1_000_000 / df_cap_prod_pop['population']

# Calcul du rapport global (métropole) pour chaque jour fractionnel en W/habitant
df_global = df_cap_prod_pop.groupby(['annee', 'jour_fractionnel']).agg({
    'cap_prod_max_thermique': 'sum',
    'cap_prod_max_hydraulique': 'sum',
    'cap_prod_max_nucleaire': 'sum',
    'eolien_p_disp': 'sum',
    'solaire_p_disp': 'sum',
    'cap_prod_max_bioenergies': 'sum',
    'population': 'sum'
}).reset_index()

# Calcul de la puissance installée globale (métropole) sur la population en W/habitant
df_global['cap_max_pop_metropole'] = (df_global['cap_prod_max_thermique'] +
                                        df_global['cap_prod_max_nucleaire'] +
                                        df_global['cap_prod_max_hydraulique'] +
                                        df_global['eolien_p_disp'] +
                                        df_global['solaire_p_disp'] +
                                        df_global['cap_prod_max_bioenergies']) * 1_000_000 / df_global['population']

# Création d'une fonction pour générer un graphique par année avec toutes les courbes
def plot_capacite_population_total(df, df_global, annee):
    df_annee = df[df['annee'] == annee]
    df_global_annee = df_global[df_global['annee'] == annee]

    # Création de la figure
    plt.figure(figsize=(12, 8))
    sns.set_theme(style="whitegrid")

    regions = df['region'].unique()
    palette = sns.color_palette("husl", len(regions) + 1)
    # Courbes régionales
    for i, region in enumerate(regions):
        df_region = df_annee[df_annee['region'] == region]
        sns.lineplot(x='jour_fractionnel', y='cap_max_pop_region', data=df_region, label=region, color=palette[i])
        # Ajout des labels à l'extrémité des courbes
        final_value = df_region[df_region['jour_fractionnel'] == df_region['jour_fractionnel'].max()]
        plt.annotate(region, 
                     xy=(final_value['jour_fractionnel'].values[0], final_value['cap_max_pop_region'].values[0]),
                     xytext=(5, 5), 
                     textcoords='offset points',
                     color=palette[i],
                     fontsize=8)

    # Courbe métropole en pointillés noirs
    sns.lineplot(x='jour_fractionnel', y='cap_max_pop_metropole', data=df_global_annee, color='k', linestyle='--', label='Métropole')
    # Ajout d'un label "Métropole" au jour fractionnel 175
    plt.annotate('Métropole', 
                 xy=(175, df_global_annee[df_global_annee['jour_fractionnel'] == 175]['cap_max_pop_metropole'].values[0]),
                 xytext=(5, 5), 
                 textcoords='offset points',
                 color='k',
                 fontsize=8,
                 ha='center')

    plt.title(f'Puissance de production électrique installée par habitant - Année {annee}')
    plt.xlabel('Jour')
    plt.ylabel('Puissance installée (cap_max_toutes_filières) / Population ( en  W / Habitant )')
    plt.grid(True)

    # Ajustement des graduations de l'axe y
    plt.gca().yaxis.set_major_locator(plt.MultipleLocator(500))

    # Suppression de la légende
    plt.legend().set_visible(False)

    filename = os.path.join(output_dir, f'cap_max_population_jour_{annee}.png')
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.show()

    # Génération des graphiques pour chaque année de 2013 à 2022
for year in range(2013, 2023):
    plot_capacite_population_total(df_cap_prod_pop, df_global, year)

In [ ]:
# Ajout d'une colonne 'cap_max_6f' dans df_cap_prod_pop correspondant à la somme des 6 capacités maximales
df_cap_prod_pop['cap_max_6f'] = (df_cap_prod_pop['cap_prod_max_thermique'] +
                                  df_cap_prod_pop['cap_prod_max_nucleaire'] +
                                  df_cap_prod_pop['cap_prod_max_hydraulique'] +
                                  df_cap_prod_pop['eolien_p_disp'] +
                                  df_cap_prod_pop['solaire_p_disp'] +
                                  df_cap_prod_pop['cap_prod_max_bioenergies']) / 2
# Création du dataframe df_cap_pop_annee
years = range(2013, 2023)
regions = df_cap_prod_pop['region'].unique()

df_cap_pop_annee = pd.DataFrame([(year, region) for year in years for region in regions], columns=['annee', 'region'])

# Ajout de la région 'metropole' pour chaque année
df_metropole = pd.DataFrame([(year, 'METROPOLE') for year in years], columns=['annee', 'region'])
df_cap_pop_annee = pd.concat([df_cap_pop_annee, df_metropole], ignore_index=True)


In [ ]:
# Calcul de la population moyenne pour chaque combinaison 'annee' et 'region'
def calculate_population_moyenne(row):
    if row['region'] != 'METROPOLE':
        return df_cap_prod_pop[(df_cap_prod_pop['annee'] == row['annee']) & (df_cap_prod_pop['region'] == row['region'])]['population'].mean()
    else:
        return df_cap_prod_pop[df_cap_prod_pop['annee'] == row['annee']].groupby('region')['population'].mean().sum()

df_cap_pop_annee['population moyenne'] = df_cap_pop_annee.apply(calculate_population_moyenne, axis=1)

# Calcul de la somme divisée par 2 (mesures de puissance relevées toutes les 30mn)des capacités maximales pour chaque 'annee' et 'region'
def calculate_cap_max_6f(row):
    if row['region'] != 'METROPOLE':
        return df_cap_prod_pop[(df_cap_prod_pop['annee'] == row['annee']) & (df_cap_prod_pop['region'] == row['region'])]['cap_max_6f'].sum()
    else:
        return df_cap_prod_pop[df_cap_prod_pop['annee'] == row['annee']]['cap_max_6f'].sum()

df_cap_pop_annee['cap_prod_max_6f'] = df_cap_pop_annee.apply(calculate_cap_max_6f, axis=1)

# Ajout d'une colonne contenant le ratio cap_prod_max_6f / population moyenne
df_cap_pop_annee['ratio_cap_pop'] = df_cap_pop_annee['cap_prod_max_6f'] / df_cap_pop_annee['population moyenne']

# Calcul de la somme des échanges physiques divisée par 2 pour chaque 'annee' et 'region'
def calculate_solde_ech_physiques(row):
    if row['region'] != 'METROPOLE':
        return df_cap_prod_pop[(df_cap_prod_pop['annee'] == row['annee']) & (df_cap_prod_pop['region'] == row['region'])]['ech_physiques'].sum() / 2 
    else:
        return df_cap_prod_pop[df_cap_prod_pop['annee'] == row['annee']]['ech_physiques'].sum() / 2

df_cap_pop_annee['solde_ech_physiques'] = df_cap_pop_annee.apply(calculate_solde_ech_physiques, axis=1)

# Ajout d'une colonne contenant le ratio solde_ech_physiques / population moyenne
df_cap_pop_annee['ratio_ech_phys'] = df_cap_pop_annee['solde_ech_physiques'] / df_cap_pop_annee['population moyenne']

In [ ]:
df_cap_pop_annee.info()

In [ ]:
# ++++++++++++++++++++++++          Capacités maximales de production / population, HISTOGRAMMES par ANNEE et par REGION

# Création d'une fonction pour tracer les histogrammes à partir de la colonne des ratios
def plot_capacite_population_annuelle(df, annee):
    df_annee = df[df['annee'] == annee]
    
    # Création de la figure
    plt.figure(figsize=(12, 8))
    sns.set_theme(style="whitegrid")
    
    regions = df_annee['region'].unique()
    x = range(len(regions))  # Crée une liste de positions correspondant au nombre de régions
    width = 0.7
    
    # Barres régionales
    for i, region in enumerate(regions):
        df_region = df_annee[df_annee['region'] == region]
        plt.bar(x[i], df_region['ratio_cap_pop'].values[0], width, label=region, color=sns.color_palette("husl", len(regions))[i])

    plt.title(f'Capacités de production annuelles installées par habitant en {annee}')
    plt.xlabel('Région')
    plt.ylabel('Rapport (cap_max_toutes_filières / Population) (Wh/habitant/an)')
    
    # Assignation des labels des régions aux graduations des abscisses
    plt.xticks(ticks=x, labels=regions, rotation=45, ha='right')

    # Ajout de la légende
    plt.legend().set_visible(False)

    plt.grid(True, axis='y')

    # Ajustement des graduations de l'axe y
    plt.gca().yaxis.set_major_locator(plt.MaxNLocator(integer=True))  # Affiche les graduations entières
    plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{int(x):,}'))  # Formatage des labels
    
    # Sauvegarde du graphique
    filename = os.path.join(output_dir, f'cap_max_population_annuelle_{annee}.png')
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.show()

# Génération des histogrammes pour chaque année de 2013 à 2022
for year in range(2013, 2023):
    plot_capacite_population_annuelle(df_cap_pop_annee, year)

In [ ]:
# HISTOGRAMMES RATIOS -LARGEURS PROPORTIONNELS A LA POPULATION ET INSERTION DES NIVEAUX D'ECHANGES PHYSIQUES

In [ ]:
# Création du répertoire de sortie 
output_dir = "Cap_max_pop_an_prop_EC"
os.makedirs(output_dir, exist_ok=True)

# Création d'une fonction pour tracer les histogrammes à partir de la colonne des ratios
def plot_capacite_population_annuelle(df, annee):
    df_annee = df[df['annee'] == annee]
    
    # Séparation des données de 'METROPOLE' et des autres régions
    df_regions = df_annee[df_annee['region'] != 'METROPOLE']
    df_metropole = df_annee[df_annee['region'] == 'METROPOLE']
    
    # Tri des régions par ordre alphabétique
    df_regions = df_regions.sort_values(by='region')
    regions = df_regions['region'].unique()
    
    # Création de la figure
    plt.figure(figsize=(12, 8))
    sns.set_theme(style="whitegrid")
    
    x = range(len(regions))  # Crée une liste de positions correspondant au nombre de régions
    
    # Normalisation des populations pour ajuster les largeurs (sans 'METROPOLE')
    populations = df_regions['population moyenne'].values
    normalized_populations = populations / populations.max()  # Normalisation par la population maximale
    width_factor = 1.3  # Facteur de largeur de base
    
    # Barres régionales avec largeurs proportionnelles à la population (sans 'METROPOLE')
    for i, region in enumerate(regions):
        df_region = df_regions[df_regions['region'] == region]
        ratio_cap_pop = df_region['ratio_cap_pop'].values[0]
        ratio_ech_phys = df_region['ratio_ech_phys'].values[0]
        
        # Largeur proportionnelle pour le bâton intérieur
        inner_width_factor = 0.5  # Facteur de largeur pour le bâton intérieur
        inner_width = width_factor * normalized_populations[i] * inner_width_factor
        
        # Tracé des barres
        plt.bar(x[i], ratio_cap_pop, 
                width=width_factor * normalized_populations[i],  # Largeur proportionnelle à la population
                label=region, 
                color=sns.color_palette("husl", len(regions))[i])
        
        # Tracé du bâton intérieur avec hachures
        plt.bar(x[i], abs(ratio_ech_phys), 
                width=inner_width, 
                bottom=0 if ratio_ech_phys >= 0 else ratio_ech_phys,  # Déplacement du bas du bâton intérieur selon la valeur
                color='cyan' if ratio_ech_phys >= 0 else 'yellow',  # Couleur en fonction du signe
                alpha=0.4,  # Transparence pour mieux voir les deux bâtons
                edgecolor='black',
                hatch='/' if ratio_ech_phys >= 0 else '\\')  # Hachures selon le signe

    # Tracé de la ligne pour 'METROPOLE'
    if not df_metropole.empty:
        metropole_ratio = df_metropole['ratio_cap_pop'].values[0]
        plt.axhline(y=metropole_ratio, color='red', linestyle='--', label='METROPOLE')
        
        # Ajout du label au bout de la ligne
        plt.text(len(regions) - 0.5, metropole_ratio, 'METROPOLE', color='red', va='center')

    plt.title(f'Ratios des Capacités de production annuelles installées, et soldes des échanges physiques (hachures)  par habitant en {annee}')
    plt.xlabel('Région')
    plt.ylabel('Ratios cap_max_6f/Pop et ech_physiques/Pop) (Wh/habitant/an)')
    
    # Assignation des labels des régions aux graduations des abscisses
    plt.xticks(ticks=x, labels=regions, rotation=45, ha='right')

    # Ajout de l'encadré avec la légende pour 'ratio_ech_phys'
    handles = [
        patches.Patch(color='cyan', hatch='/', label='Solde Importateur'),
        patches.Patch(color='yellow', hatch='\\', label='Solde Exportateur')
    ]
    plt.legend(handles=handles, loc='best', title='Légende', bbox_to_anchor=(1.05, 1), bbox_transform=plt.gcf().transFigure, fancybox=True, shadow=True, ncol=1)

    plt.grid(True, axis='y')

    # Ajustement des graduations de l'axe y
    plt.gca().yaxis.set_major_locator(plt.MaxNLocator(integer=True))  # Affiche les graduations entières
    plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{int(x):,}'))  # Formatage des labels

    # Ajout de l'encadré en haut à droite
    plt.text(0.98, 0.98, 'Légende : -- Ratio des échanges physiques', ha='right', va='top', transform=plt.gca().transAxes)

    # Ajout d'une note en bas du graphique
    plt.text(0.85, 0.85, 'Largeur des bâtons proportionnelles à la population', 
             ha='center', va='center', transform=plt.gca().transAxes, fontsize=10, color='black', bbox=dict(facecolor='white', edgecolor='none', boxstyle='round,pad=0.5'))
    
    # Enregistrement du graphique
    plt.savefig(f"{output_dir}/ratio_cap_pop_annee_{annee}.png", dpi=300, bbox_inches='tight')
    plt.show()

# Génération des histogrammes pour chaque année de 2013 à 2022
for year in range(2013, 2023):
    plot_capacite_population_annuelle(df_cap_pop_annee, year)

In [ ]:
#==========================  Affichage des capacités maximales de chaque filière de production au fil des jours ===========

In [ ]:
#  ------------------- Affichage de toutes les filières par région sur les mêmes visualisations -------------

# Liste des filières
filières = ['cmax_thermique_rte', 'cmax_nucleaire_rte', 'cmax_eolien_rte', 'cmax_solaire_rte', 'cmax_hydraulique_rte', 'cmax_bioenergies_rte', 'import']
colors = ['blue', 'green', 'red', 'orange', 'purple', 'brown', 'black']

# Fonction pour ajouter les barres de saisons
def barre_saisons(ax):
    saisons = {'Hiver': (1, 79),
               'Printemps': (80, 171),
               'Été': (172, 265),
               'Automne': (266, 354),
               'Hiver (suite)': (355, 365)}
    colors = ['lightblue', 'lightgreen', 'lightyellow', 'orange', 'lightblue']
    for (saison, (start, end)), color in zip(saisons.items(), colors):
        ax.axvspan(start, end, color=color, alpha=0.3)
        ax.text((start + end) / 2, 105, saison, ha='center', va='top', fontsize=10, color='gray')

# Liste des régions
regions = df_cap_prod['region'].unique()

# Pour chaque région
for region in regions:
    # Création d'une figure avec 3 sous-graphiques
    fig, axs = plt.subplots(3, 1, figsize=(20, 30), sharey=True, gridspec_kw={'height_ratios': [3, 3, 3]})
    fig.subplots_adjust(hspace=0.2)  # Espace entre les sous-graphiques
    handles_labels = []
    
    # Pour chaque année
    for i, annee in enumerate([2020, 2021, 2022]):
        # Sélection des données pour la région et l'année
        data = df_cap_prod[(df_cap_prod['region'] == region) & (df_cap_prod['annee'] == annee)]
                     
        # Pour chaque filière
        for j, filière in enumerate(filières):
            # Groupement des données par 'jour_numero'
            grouped_data = data.groupby('jour_numero')
            
            # Calcul de la capacité maximale moyenne pour chaque 'jour_numero'
            c_max = grouped_data[filière].mean()            
            # Ajout de la courbe au graphique sans label pour éviter duplication
            line, = axs[i].plot(grouped_data['jour_numero'].first(), c_max, label=filière, color=colors[j])
            if i == 0:  # Ajouter les handles et labels pour la légende une seule fois
                handles_labels.append((line, filière))
                 
        # Ajout des titres et étiquettes
        axs[i].set_title(f'Capacité maximale journalière moyenne de production pour la région {region} en {annee}')
        axs[i].set_ylabel("Capacité maximale journalière moyenne (MW)")
        axs[i].set_xlim(0, 370)  # Fixation des limites des abscissesordonnées
        #axs[i].set_ylim(0, 110)  # Fixation des limites des ordonnées
        axs[i].grid(True, which='both', linestyle='--', linewidth=0.5, color='gray')  # Ajouter une grille gris pâle
        barre_saisons(axs[i])
    
    # Affichage des étiquettes des graduations des abscisses sur tous les sous-graphiques
    for ax in axs:
        ax.set_xticks(range(0, 366, 20))
        ax.set_xticklabels(range(0, 366, 20), rotation=45)
        ax.set_xlabel("Numéro de jour de l'année")
    
    # Création d'une légende unique
    handles, labels = zip(*handles_labels)
    fig.legend(handles, labels, loc='lower center', ncol=7, title="Filières énergétiques")
    plt.subplots_adjust(bottom=0.1)
    
    # Sauvegarde du graphique en PNG
    plt.savefig(f'c_max_{region}.png', dpi=300)
    plt.show()

In [ ]:
#  ------------------- Affichage du graphique pour la métropole -------------

In [ ]:
# Liste des colonnes spécifiques
#selected_columns_tch = ['tch_thermique_rte', 'tch_nucleaire_rte', 'tch_eolien_rte', 'tch_solaire_rte', 'tch_hydraulique_rte','tch_bioenergies_rte','date_heure']

selected_columns_prod = ['thermique', 'nucleaire', 'eolien', 'solaire', 'hydraulique','bioenergies','date_heure']
selected_columns_c_max = ['cmax_thermique_rte', 'cmax_nucleaire_rte', 'cmax_eolien_rte', 'cmax_solaire_rte', 'cmax_hydraulique_rte',
                    'cmax_bioenergies_rte','date_heure']


# Conversion de la colonne "date_heure" en datetime
df_cap_prod['date_heure'] = pd.to_datetime(df_cap_prod['date_heure'], format='%Y-%m-%d %H:%M:%S')

# Sélection des colonnes à afficher pour toutes les régions
df_selected_prod = df_cap_prod[selected_columns_prod]
df_selected_c_max = df_cap_prod[selected_columns_c_max]


# Calcul des sommes des capacités maximales et des productions effectives de toutes les filières par, par semaine pour toutes les régions
df_semaine_metro_sum = df_selected_c_max.resample('W', on='date_heure').sum()

# Calcul du nombre de points de données par semaine pour toutes les régions
nb_points_par_semaine = df_selected_c_max.resample('W', on='date_heure').size()

# Calcul de la moyenne en divisant la somme par le nombre de points de données
df_semaine_metro_mean = df_semaine_metro_sum.div(nb_points_par_semaine, axis=0)
#    ---------------------------------------------------------------------------

# Création de la figure
fig, ax = plt.subplots(figsize=(12, 6), dpi = 300)

# Tracé des courbes pour chaque colonne
for col in selected_columns_c_max[:-1]:  # Exclusion de 'date_heure'
    ax.plot(df_semaine_metro_mean.index, df_semaine_metro_mean[col], label=col)

# Formatage des date pour l'axe des abscisses
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))  # Etiquettes tous les 3 mois
ax.xaxis.set_minor_locator(mdates.MonthLocator())  # Etiquettes secondaires tous les mois
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  # Formatage des étiquettes de date

# Ajout des légendes, titres et étiquettes
ax.legend(loc='upper left', bbox_to_anchor=(1, 1), title="Filières énergétiques")
ax.set_title("Taux de Couvertures (tco moyennes hebdomadaires) des flux électriques pour la Métropole")
ax.set_xlabel("Date")
ax.set_ylabel("tco (%)")

# Rotation des dates pour une meilleure lisibilité
plt.xticks(rotation=90)

# Réduction de la taille de police des étiquettes de l'axe des abscisses
ax.tick_params(axis='x', labelsize=10)

# Ajout d'une grille avec des traits gris pâles
ax.grid(True, which='both', linestyle='--', linewidth=0.5, color='grey', alpha=0.7)

# Ajustement de la mise en page pour s'assurer que tout est visible
plt.tight_layout()

# Ajout d'une ligne horizontale à y=100 %
plt.axhline(y=100, color='red', linestyle='--', linewidth=1, label='Seuil tco = 100')

# Affichage du graphique
plt.show()

In [ ]:
#        =================>    Affichages des graphiques par région

# Liste des colonnes à afficher
selected_columns = ['tco_thermique', 'tco_nucleaire', 'tco_eolien', 'tco_solaire', 'tco_hydraulique', 
                    'tco_pompage', 'tco_bioenergies', 'tco_export', 'tco_import','tco_stockage_batterie', 
                    'tco_destockage_batterie','date_heure']

# Boucle pour afficher toutes les régions
for region in df_cap_prod['region'].unique():
    # Filtrage par région
    df_region = df_cap_prod[df_cap_prod['region'] == region]

    # Conversion de la colonne "date_heure" en datetime
    df_region['date_heure'] = pd.to_datetime(df_region['date_heure'], format='%Y-%m-%d %H:%M:%S')

    # Sélection des colonnes à afficher
    df_region_tco = df_region[selected_columns]

    # Calcul de la somme par semaine
    df_region_semaine_sum = df_region_tco.resample('W', on='date_heure').sum()

    # Calcul du nombre de points de données par semaine
    data_points_par_semaine = df_region_tco.resample('W', on='date_heure').size()

    # Calcul de la moyenne hebdomadaire
    df_region_semaine_mean = df_region_semaine_sum.div(data_points_par_semaine, axis=0)

    # Création de la figure
    fig, ax = plt.subplots(figsize=(12, 6), dpi = 300)

    # Tracé des courbes pour chaque colonne
    for col in selected_columns[:-1]:  # Exclusion de 'date_heure' pour le tracé
        ax.plot(df_region_semaine_mean.index, df_region_semaine_mean[col], label=col)

    # Formatage des dates pour l'axe des abscisses
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))  # Graduation principale tous les 3 mois
    ax.xaxis.set_minor_locator(mdates.MonthLocator())  # Graduations secondaires tous les mois
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  # Formatage des étiquettes de date

    # Délimitation de l'axe des abscisses
    ax.set_xlim(pd.Timestamp('2013-01-01'), df_region_semaine_mean.index.max())
    
    # Ajout des légendes, titres et étiquettes
    ax.legend(loc='upper left', bbox_to_anchor=(1, 1), title="Filières énergétiques")
    ax.set_title(f"Taux de Couvertures (tco moyennes hebdomadaires) des flux électriques en {region}")
    # Décalage du titre vers le haut
    ax.title.set_position([.5, 1.05])
    ax.set_xlabel("Date")
    ax.set_ylabel("tco (%)")

    # Rotation des dates pour une meilleure lisibilité
    plt.xticks(rotation=90)
    
    # Réduction de la taille de police des étiquettes de l'axe des abscisses
    ax.tick_params(axis='x', labelsize=9) 
    
    # Ajout d'une grille avec des traits gris pâles
    ax.grid(True, which='major', linestyle='--', linewidth=0.5, color='grey', alpha=0.7)
    
    # Ajustement de la mise en page pour s'assurer que tout est visible
    plt.tight_layout()
    
    # Ajout d'une ligne horizontale à y=100 %
    ax.axhline(y=100, color='red', linestyle='--', linewidth=1, label='Seuil tco = 100')

    # Affichage du graphique
    plt.show()

In [ ]:
# ------------------------------ TCO 'toutes filières de production'/pompage/ export / import / stockage batterie / déstockage batterie ----------------------------
# Affichage de l'évolution des taux de couvertures régionales en fonction du temps, par région
# Le volume de données étant trop important pour obtenir des grapphiques lisibles, il est choisi de n'afficher que les moyennes
# des tco calculées par semaine

In [ ]:
#   =================>    Affichage du graphique pour la métropole

# Liste des colonnes spécifiques
selected_columns = ['tco_pompage', 'tco_export','tco_import', 'tco_stockage_batterie', 
                    'tco_destockage_batterie', 'date_heure']

# Conversion de la colonne "date_heure" en datetime
df_cap_prod['date_heure'] = pd.to_datetime(df_cap_prod['date_heure'], format='%Y-%m-%d %H:%M:%S')

# Sélection des colonnes à afficher pour toutes les régions
df_selected = df_cap_prod[selected_columns]

# Calcul de la somme par semaine pour toutes les régions
df_semaine_metro_sum = df_selected.resample('W', on='date_heure').sum()

# Calcul du nombre de points de données par semaine pour toutes les régions
nb_points_par_semaine = df_selected.resample('W', on='date_heure').size()

# Calcul de la moyenne en divisant la somme par le nombre de points de données
df_semaine_metro_mean = df_semaine_metro_sum.div(nb_points_par_semaine, axis=0)
#    ---------------------------------------------------------------------------

# Création de la figure
fig, ax = plt.subplots(figsize=(12, 6), dpi = 300)

# Tracé des courbes pour chaque colonne
for col in selected_columns[:-1]:  # Exclusion de 'date_heure'
    ax.plot(df_semaine_metro_mean.index, df_semaine_metro_mean[col], label=col)

# Formatage des date pour l'axe des abscisses
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))  # Etiquettes tous les 3 mois
ax.xaxis.set_minor_locator(mdates.MonthLocator())  # Etiquettes secondaires tous les mois
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  # Formatage des étiquettes de date

# Ajout des légendes, titres et étiquettes
ax.legend(loc='upper left', bbox_to_anchor=(1, 1), title="Filières énergétiques")
ax.set_title("Taux de Couvertures (tco moyennes hebdomadaires) des flux électriques pour la Métropole - Regroupements")
ax.set_xlabel("Date")
ax.set_ylabel("tco (%)")

# Rotation des dates pour une meilleure lisibilité
plt.xticks(rotation=90)

# Réduction de la taille de police des étiquettes de l'axe des abscisses
ax.tick_params(axis='x', labelsize=10)

# Ajout d'une grille avec des traits gris pâles
ax.grid(True, which='both', linestyle='--', linewidth=0.5, color='grey', alpha=0.7)

# Ajustement de la mise en page pour s'assurer que tout est visible
plt.tight_layout()

# Ajout d'une ligne horizontale à y=100 %
plt.axhline(y=100, color='red', linestyle='--', linewidth=1, label='Seuil tco = 100')

# Affichage du graphique
plt.show()

In [ ]:
#        =================>    Affichages des graphiques par région

In [ ]:
# Liste des colonnes à afficher
selected_columns = ['tco_pompage', 'tco_export','tco_import', 'tco_stockage_batterie', 
                    'tco_destockage_batterie', 'date_heure']

# Boucle pour afficher toutes les régions
for region in df_cap_prod['region'].unique():
    # Filtrage par région
    df_region = df_cap_prod[df_cap_prod['region'] == region]

    # Conversion de la colonne "date_heure" en datetime
    df_region['date_heure'] = pd.to_datetime(df_region['date_heure'], format='%Y-%m-%d %H:%M:%S')

    # Sélection des colonnes à afficher
    df_region_tco = df_region[selected_columns]

    # Calcul de la somme par semaine
    df_region_semaine_sum = df_region_tco.resample('W', on='date_heure').sum()

    # Calcul du nombre de points de données par semaine
    data_points_par_semaine = df_region_tco.resample('W', on='date_heure').size()

    # Calcul de la moyenne hebdomadaire
    df_region_semaine_mean = df_region_semaine_sum.div(data_points_par_semaine, axis=0)

    # Création de la figure
    fig, ax = plt.subplots(figsize=(12, 6), dpi = 300)

    # Tracé des courbes pour chaque colonne
    for col in selected_columns[:-1]:  # Exclusion de 'date_heure' pour le tracé
        ax.plot(df_region_semaine_mean.index, df_region_semaine_mean[col], label=col)

    # Formatage des dates pour l'axe des abscisses
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))  # Graduation principale tous les 3 mois
    ax.xaxis.set_minor_locator(mdates.MonthLocator())  # Graduations secondaires tous les mois
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  # Formatage des étiquettes de date

    # Délimitation de l'axe des abscisses
    ax.set_xlim(pd.Timestamp('2013-01-01'), df_region_semaine_mean.index.max())
    
    # Ajout des légendes, titres et étiquettes
    ax.legend(loc='upper left', bbox_to_anchor=(1, 1), title="Filières énergétiques")
    ax.set_title(f"Taux de Couvertures (tco moyennes hebdomadaires) des flux électriques en {region} -  - Regroupements")
    # Décalage du titre vers le haut
    ax.title.set_position([.5, 1.05])
    ax.set_xlabel("Date")
    ax.set_ylabel("tco (%)")

    # Rotation des dates pour une meilleure lisibilité
    plt.xticks(rotation=90)
    
    # Réduction de la taille de police des étiquettes de l'axe des abscisses
    ax.tick_params(axis='x', labelsize=9) 
    
    # Ajout d'une grille avec des traits gris pâles
    ax.grid(True, which='major', linestyle='--', linewidth=0.5, color='grey', alpha=0.7)
    
    # Ajustement de la mise en page pour s'assurer que tout est visible
    plt.tight_layout()
    
    # Ajout d'une ligne horizontale à y=100 %
    ax.axhline(y=100, color='red', linestyle='--', linewidth=1, label='Seuil tco = 100')

    # Affichage du graphique
    plt.show()

In [ ]:
# ------------------- ANALYSE DES CAPACITES DE PRODUCTION MAXIMALES ET DISPONIBLES D'ENERGIE ANNUELLES --------

In [ ]:
# CAPACITES MAXIMALE  PAR REGION ET PAR ANNEE


# Création du répertoire de sauvegarde des graphiques
output_dir = "Hist_cap_max_regions"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Dictionnaire pour les noms des filières
filiere_cols0 = {
    'Thermique': 'thermique_p_disp',
    'Nucléaire': 'nucleaire_p_disp',
    'Hydraulique': 'hydraulique_p_disp',
    'Bioénergies': 'bioenergies_p_disp',
    'Eolien': 'eolien_p_disp',
    'Solaire': 'solaire_p_disp'
}

# Liste des régions uniques
regions = df_cap_prod['region'].unique()

# Boucle sur chaque région
for region in regions:
    # Filtrage des données pour la région en cours
    df_region = df_cap_prod[df_cap_prod['region'] == region]
    
    # Liste pour stocker les flux annuels par région
    flux_annuel = []
    annees = range(2013, 2023)
    
    # Boucle sur chaque année
    for annee in annees:
        # Filtrage des données pour l'année en cours
        df_annee = df_region[df_region['annee'] == annee]
        
        # Calcul des capacités annuelles en MWh pour chaque type de production
        flux = {}
        for filiere, colonne in filiere_cols0.items():
            flux[filiere] = (df_annee[colonne] * 0.5).sum()  # Multiplie par 0.5 pour prendre en compte la demi-heure
        
        # Ajout du flux annuel au résultat
        flux_annuel.append(flux)
    
    # Création d'un DataFrame pour stocker les résultats
    df_flux_annuel = pd.DataFrame(flux_annuel, index=annees)
    
    # Couleurs pour chaque type de production
    couleurs = ['purple', 'gold', 'aquamarine', 'orange', 'blue', 'green', 'crimson']
    
    # Largeur des barres
    largeur_barres = 0.7 / len(df_flux_annuel.columns)
    
    # Tracage des histogrammes pour chaque année
    plt.figure(figsize=(12, 8), dpi=300)
    for i, filiere in enumerate(df_flux_annuel.columns):
        positions = np.arange(len(df_flux_annuel)) + i * largeur_barres
        plt.bar(positions, df_flux_annuel[filiere].divide(1000000), width=largeur_barres, color=couleurs[i], label=filiere)
    
    # Configuration du graphique
    plt.xlabel('Année')
    plt.ylabel('Production annuelle (TWh) / an')
    plt.title(f"Bilans annuels des capacités de production d'électricité pour la région {region}")
    plt.xticks(np.arange(len(df_flux_annuel)) + largeur_barres * (len(df_flux_annuel.columns) - 1) / 2, df_flux_annuel.index)
    
    # Ajout de la légende avec les noms des filières
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), fancybox=True, shadow=True, ncol=6)
    
    # Ajout de lignes verticales discrètes pour délimiter les groupes de barres
    for pos in np.arange(1, len(df_flux_annuel)):
        plt.axvline(x=pos - 0.5, color='gray', linestyle='--', linewidth=0.5, alpha=0.7)
    
    # Grille
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Réduction de la taille de police des étiquettes de l'axe des abscisses
    plt.tick_params(axis='x', labelsize=10)
    
    # Ajustement de la mise en page
    plt.tight_layout()
    
    # Sauvegarde du graphique
    filename = f"hist_cap_max_{region}.png"
    plt.savefig(os.path.join(output_dir, filename), dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
# CAPACITES MAXIMALE  METROPOLE

# Création du répertoire de sauvegarde des graphiques

# Calcul des capacités de production d'énergie annuelles pour chaque année et chaque type de production, pour l'ensemble des régions, sans prise en compte des indisponibilités
# à l'exception des filière éolienne et solaire pour lesquelles les capacités de production maximales n'ont pas été trouvées.

In [ ]:
# Calcul des capacités de production d'énergie annuelles pour chaque année
annees = range(2013, 2023)
flux_annuel = []

for annee in annees:
    # Filtrage des données pour l'année en cours
    df_annee = df_cap_prod[df_cap_prod['annee'] == annee]
    
    # Calcul des capacités annuelles en MWh pour chaque type de production
    flux = {}
    for filiere, colonne in filiere_cols0.items():
        flux[filiere] = (df_annee[colonne] * 0.5).sum()  # Prendre en compte la demi-heure
    
    # Ajout du flux annuel au résultat
    flux_annuel.append(flux)

# Création d'un DataFrame pour stocker les résultats
df_flux_annuel = pd.DataFrame(flux_annuel, index=annees)

# Couleurs pour chaque type de production
couleurs = ['purple', 'gold', 'aquamarine', 'orange', 'blue', 'green', 'crimson']

# Largeur des barres
largeur_barres = 0.7 / len(df_flux_annuel.columns)

# Tracage des histogrammes pour chaque année
plt.figure(figsize=(12, 8), dpi=300)
for i, filiere in enumerate(df_flux_annuel.columns):
    positions = np.arange(len(df_flux_annuel)) + i * largeur_barres
    plt.bar(positions, df_flux_annuel[filiere].divide(1000000), width=largeur_barres, color=couleurs[i], label=filiere)

# Configuration des axes et du titre
plt.xlabel('Année')
plt.ylabel('Production annuelle (TWh) / an')
plt.title("Bilans annuels des capacités de production d'électricitée en France métropolitaine")
plt.xticks(np.arange(len(df_flux_annuel)) + largeur_barres * (len(df_flux_annuel.columns) - 1) / 2, df_flux_annuel.index)

# Positionnement de la légende sous le graphique
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), fancybox=True, shadow=True, ncol=6)

# Ajout de lignes verticales discrètes pour délimiter les groupes de barres
for pos in np.arange(1, len(df_flux_annuel)):
    plt.axvline(x=pos - 0.5, color='gray', linestyle='--', linewidth=0.5, alpha=0.7)

# Grille sur l'axe des ordonnées
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Réduction de la taille de police des étiquettes de l'axe des abscisses
plt.tick_params(axis='x', labelsize=10)

# Ajustement de la mise en page pour s'assurer que tout est visible
plt.tight_layout()

# Sauvegarde du graphique
filename = "hist_cap_max_metropole.png"
plt.savefig(os.path.join(output_dir, filename), dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# CAPACITES REELLES, AVEC PRISE EN COMPTE DES INDISPONIBILITES

In [ ]:
# Calcul des capacités de production d'énergie annuelles pour chaque année et chaque type de production, pour l'ensemble des régions, sans prise en compte des indisponibilités
annees = range(2013, 2023)
flux_annuel = []

for annee in annees:
    # Filtrage des données pour l'année en cours
    df_annee = df_cap_prod[df_cap_prod['annee'] == annee]
    
    # Nombre total d'heures dans l'année
    nombre_heures = (len(df_annee)-1)*0.5
    
    # Calcul des capacités annuelles en MWh poour 6 types de production
    flux = {}
    for colonne in ['thermique_p_disp', 'nucleaire_p_disp', 'eolien_p_disp',
                    'solaire_p_disp', 'hydraulique_p_disp', 'bioenergies_p_disp']:
         flux[colonne] = (df_annee[colonne]*0.5).sum()
    
    # Ajout du flux annuel au résultat
    flux_annuel.append(flux)

# Création d'un DataFrame pour stocker les résultats
df_flux_annuel = pd.DataFrame(flux_annuel, index=annees)

# Couleurs pour chaque type de production
couleurs = ['purple', 'gold', 'aquamarine', 'orange', 'blue','green', 'crimson']

# Largeur des barres
largeur_barres = 0.7 / len(df_flux_annuel.columns)

# Tracage des histogrammes pour chaque année
plt.figure(figsize=(12, 8), dpi = 300)
for i, colonne in enumerate(df_flux_annuel.columns):
    positions = np.arange(len(df_flux_annuel)) + i * largeur_barres
    plt.bar(positions, df_flux_annuel[colonne].divide(1000000), width=largeur_barres, color=couleurs[i], label=colonne)
    
plt.xlabel('Année')
plt.ylabel('Production annuelle (TWh) / an')
plt.title("Bilans annuels des capacités réelles de production d'électricité en France métropolitaine, AVEC les indisponibilités" )
plt.xticks(np.arange(len(df_flux_annuel)) + largeur_barres * (len(df_flux_annuel.columns) - 1) / 2, df_flux_annuel.index)

plt.legend(loc='upper right', bbox_to_anchor=(1, 1))
# Ajout de lignes verticales discrètes pour délimiter les groupes de barres
for pos in np.arange(1, len(df_flux_annuel)):
    plt.axvline(x=pos - 0.5, color='gray', linestyle='--', linewidth=0.5, alpha=0.7)

plt.grid(axis='y', linestyle='--', alpha=0.7)

# Réduction de la taille de police des étiquettes de l'axe des abscisses
plt.tick_params(axis='x', labelsize=10)

# Ajustement de la mise en page pour s'assurer que tout est visible
plt.tight_layout()
plt.show()


In [ ]:
# CALCUL DES TAUX DE DISPONIBILITE DES FILIERES THERMIQUE, NUCLEAIRE, HYDRAULIQUE ET BIOENERGIES
# Liste des filières et de leurs colonnes correspondantes
filiere_cols = {
    'thermique': ('thermique_p_disp', 'cap_prod_max_thermique'),
    'nucleaire': ('nucleaire_p_disp', 'cap_prod_max_nucleaire'),
    'hydraulique': ('hydraulique_p_disp', 'cap_prod_max_hydraulique'),
    'bioenergies': ('bioenergies_p_disp', 'cap_prod_max_bioenergies')
}

# Fonction pour calculer les taux de disponibilité
def calcul_disponibilite(df, filiere, disp_col, cap_col):
    df_grouped = df.groupby(['region', 'annee', 'jour_numero'])[[disp_col, cap_col]].sum().reset_index()
    df_grouped[f'taux_dispo_{filiere}'] = (df_grouped[disp_col] / df_grouped[cap_col]) * 100
    # Si le taux de disponibilité est inférieur à 0, on le fixe à 0
    df_grouped[f'taux_dispo_{filiere}'] = df_grouped[f'taux_dispo_{filiere}'].clip(lower=0)
    return df_grouped[['region', 'annee', 'jour_numero', f'taux_dispo_{filiere}']]

# Calcul des taux de disponibilité pour chaque filière
df_dispo = {}
for filiere, (disp_col, cap_col) in filiere_cols.items():
    df_dispo[filiere] = calcul_disponibilite(df_cap_prod, filiere, disp_col, cap_col)

# Calcul des taux de disponibilité pour toutes les filières combinées
df_combined = df_cap_prod.copy()
df_combined['region_p_disp'] = df_combined[[disp_col for disp_col, _ in filiere_cols.values()]].sum(axis=1)
df_combined['region_p_max'] = df_combined[[cap_col for _, cap_col in filiere_cols.values()]].sum(axis=1)
df_dispo['toutes_filieres'] = calcul_disponibilite(df_combined, 'toutes_filieres', 'region_p_disp', 'region_p_max')

# Fonction pour ajouter les barres de saisons
def barre_saisons(ax):
    saisons = {
        'Hiver': (1, 79),
        'Printemps': (80, 171),
        'Été': (172, 265),
        'Automne': (266, 354),
        'Hiver (suite)': (355, 365)
    }
    colors = ['lightblue', 'lightgreen', 'lightyellow', 'orange', 'lightblue']
    for (saison, (start, end)), color in zip(saisons.items(), colors):
        ax.axvspan(start, end, color=color, alpha=0.3)
        ax.text((start + end) / 2, -5, saison, ha='center', va='top', fontsize=10, color='gray')

# Fonction pour créer et sauvegarder les graphiques
def plot_disponibilite(df, filiere, annee, regions_colors):
    fig, ax = plt.subplots(figsize=(15, 10))
    for region, color in regions_colors.items():
        df_region = df[(df['region'] == region) & (df['annee'] == annee)]
        ax.plot(df_region['jour_numero'], df_region[f'taux_dispo_{filiere}'], label=region, color=color)
    ax.set_xlabel('Numéro du jour')
    ax.set_ylabel('Taux de disponibilité (%)')
    ax.set_title(f'Taux de disponibilité - {filiere.capitalize()} - Année {annee}', pad=40)
    ax.legend(title='Région')
    ax.grid(True, which='both', linestyle='--', linewidth=0.5, color='lightgrey')
    ax.set_xticks(range(0, 366, 10))
    # Fixation des limites de l'axe des ordonnées entre 0 et 100
    ax.set_ylim(0, 100)
    
    barre_saisons(ax)
    plt.tight_layout()
    plt.savefig(f'taux_disp_{filiere}_{annee}.png', dpi=300)
    plt.show()

# Définition des couleurs pour chaque région
regions = df_cap_prod['region'].unique()
colors = sns.color_palette('tab10', len(regions))
regions_colors = dict(zip(regions, colors))

# Génération des graphiques
for filiere in df_dispo:
    for annee in range(2013, 2023):
        plot_disponibilite(df_dispo[filiere], filiere, annee, regions_colors)

In [ ]:
# ------------------------- MATRICES DE CORRELATIONS ENTRE TCO REGIONAUX

# Liste des colonnes tco à analyser
tco_cols = [
    'tco_thermique', 'tco_nucleaire', 'tco_eolien', 'tco_solaire',
    'tco_hydraulique', 'tco_pompage', 'tco_bioenergies', 'tco_export',
    'tco_import', 'tco_stockage_batterie', 'tco_destockage_batterie'
]

# Liste des régions uniques
regions = df_cap_prod['region'].unique()

# Affichage de la matrice de corrélation pour chaque région
for region in regions:
    df_region = df_cap_prod[df_cap_prod['region'] == region]

    # Calcul de la matrice de corrélation sur les colonnes TCO
    corr_matrix = df_region[tco_cols].corr()

    # Affichage de la heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
    plt.title(f"Matrice de corrélation des variables TCO - Région : {region}")
    plt.tight_layout()
    plt.show()

In [ ]:
print(" FIN DU CODE *****  BRAVO POUR VOTRE PATIENCE ****")